Script to complete connect assignments. It is a bit old and messy, but I don't have a use for it anymore so there's no point in fixing / cleaning it up.

Requires .env file to automatically log in correctly, format as followed:
    CLIENT_USER={username here, no brackets}
    CLIENT_PASS={password, no brackets}

In [50]:
'''
Imports libraries and .env file
'''

from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as cond

from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, TimeoutException, NoSuchElementException


from webdriver_manager.chrome import ChromeDriverManager

from dotenv import load_dotenv
from pathlib import Path

import os
import regex as re
import time

path = Path(os.getcwd() +"/.env") # CHANGE ON WINDOWS
load_dotenv(dotenv_path = path)

False

In [2]:
'''
Functions that will be called
'''

def find_qna(question, mcq):
    mcq.seek(0)
    for lines in mcq.readlines():
        lines = re.split('=====', lines)
        if str.strip(lines[0]) == question:
            ret_list = list(map(lambda x: re.sub('\n', '', x), lines[1:]))
            
            if re.search(r'\sor\s', ret_list[0]):
                # ret_list = list(ret_list)
                x = re.match(r'\w*[^ ,]', ret_list[0])
                return x.group()
            return ret_list

def get_prompt(driver):
    prompt = WebDriverWait(driver, 10).until(
                cond.element_to_be_clickable(
                    (By.CLASS_NAME, 'prompt')
                )
            ).text
    return prompt

def get_question_type(driver):
    WebDriverWait(driver, 10).until(
        cond.text_to_be_present_in_element(
            (By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/div/div/div[1]/awd-probe-navigation/div/div[1]/awd-probe/avalon-probe-renderer/div/div/h2'),
            'Question'   
        )
    )
    
    q_type = WebDriverWait(driver, 10).until(
        cond.element_to_be_clickable(
            (By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/div/div/div[1]/awd-probe-navigation/div/div[1]/awd-probe/avalon-probe-renderer/div/div/h2')
        )
    ).text
    
    return q_type

def check_alert(driver):
    alert_xpath = '//*[@id="root-container"]/div[2]/div/awd-main-container/div/div/div/div[1]/awd-probe-navigation/div/div[1]/awd-probe/avalon-probe-renderer/div/div/div/div/div/mhe-alert/div'
    if driver.find_elements(By.XPATH, alert_xpath):
        try:
            to_review = WebDriverWait(driver, 10).until(
                cond.element_to_be_clickable(
                    (By.XPATH,'//*[@id="root-container"]/div[2]/div/awd-main-container/div/div/div/div[1]/awd-probe-navigation/div/div[2]/awd-lr-buttons-tray/div/div/fieldset/div/button/span/span[2]')
                )
            )
            to_review.click()
        except:
            to_review = WebDriverWait(driver, 10).until(
                cond.element_to_be_clickable(
                    (By.XPATH,'//*[@id="root-container"]/div[2]/div/awd-main-container/div/div/div/div[1]/awd-probe-navigation/div/div[2]/awd-lr-buttons-tray/div/div/fieldset/div/button/span/span[2]')
                )
            )
            driver.execute_script('arguments[0].click();', to_review)


        to_quest_btn = WebDriverWait(driver, 20).until(
            cond.element_to_be_clickable(
                (By.XPATH, '//*[@id="root-container"]/div/div/awd-main-container/div/awd-navigation-bar/div/awd-reading-button-bar/button')
            )
        )
        to_quest_btn.click()


def store_qna_mcq(driver, question, mcq):
    first_clickable = WebDriverWait(driver, 10).until(
        cond.element_to_be_clickable(
            (By.CLASS_NAME, 'choice-row')
        )
    )
    first_clickable.click()

    if re.search(r'(true)|(false)', first_clickable.text.lower()):
        first_clickable = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/div/div/div[1]/awd-probe-navigation/div/div[1]/awd-probe/avalon-probe-renderer/div/aa-air-item/div/div/div/div/div/div/div/div/div/div[2]/div[1]/div/mhe-radio-button/label')
        first_clickable.click()

        submit = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div[1]/awd-confidence-buttons/div/fieldset/div/button[2]')
        submit.click()

        ans_box = driver.find_element(By.CLASS_NAME, 'correct-answer-container')
        ans = ans_box.find_element(By.CLASS_NAME, 'answer')
        ret_str = question + '=====' + ans.text + '\n'
        mcq.write(ret_str)
    else:        
        submit = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div[1]/awd-confidence-buttons/div/fieldset/div/button[2]')
        submit.click()
        
        ans_box = driver.find_element(By.CLASS_NAME, 'correct-answer-container')
        ans = ans_box.find_elements(By.CLASS_NAME, 'choice-row')
    
        ret_str = question
        for i in range(len(ans)):
            ret_str += '=====' + ans[i].text
        ret_str += '\n'
        mcq.write(ret_str)



def store_question_blanks(driver, question, bq):
    blanks = driver.find_elements(By.CLASS_NAME, 'fitb-input')
    for i, j in enumerate(blanks):
        j.send_keys(i+1)
    
    submit = WebDriverWait(driver, 2).until(
        cond.element_to_be_clickable(
            (By.XPATH,'//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div[1]/awd-confidence-buttons/div/fieldset/div/button[2]')
        )
    )
    submit.click() 
    
    ans_key = WebDriverWait(driver, 2).until(
        cond.element_to_be_clickable(
            (By.CLASS_NAME, 'correct-answer-container' )
        )
    )
    ans_key = ans_key.find_elements(By.CLASS_NAME, 'correct-answers')

    ret_str = ''
    for i in ans_key:
        ret_str += '=====' + re.sub(r'Blank [1-9]: ', '', i.text)
    
    bq.write(question + ret_str + '\n\n')

# def answer_question_blanks(driver):
    # blanks = driver.find_elements(By.CLASS_NAME, 'fitb-input')
    

In [3]:
def solve_mcq_questions(driver, question):
    with open('current_mcq.text', 'a+') as mcq:
        mcq.seek(0)
        prompt = get_prompt(driver)

        if question in mcq.read():
            ans_key = find_qna(question, mcq)

            choices = driver.find_elements(By.CLASS_NAME, 'choice-row')
            for x in choices:
                if str.strip(x.text) in ans_key:
                    if re.search(r'(true)|(false)', ans_key[0].lower()):
                        x = x.find_element(By.CLASS_NAME, 'ahe-ui-radio')
                    x.click()
            submit = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div[1]/awd-confidence-buttons/div/fieldset/div/button[2]')
            submit.click()
        else:
            store_qna_mcq(driver, question, mcq)

def solve_blank_questions(driver, question):
    with open('current_blanks.text', 'a+') as bq:
        bq.seek(0)
        if question in bq.read():
            blanks = driver.find_elements(By.CLASS_NAME, 'fitb-input')
            ans_key = find_qna(question, bq)
            ans_key = zip(blanks, ans_key)

            for i, j in ans_key:
                i.send_keys(j)
            submit = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div[1]/awd-confidence-buttons/div/fieldset/div/button[2]')
            submit.click()
        else:
            store_question_blanks(driver, question, bq)


In [4]:
# used for troubleshooting, answers a single question

def answer_once(question, q_type):
        
    end_early = False
    valid_types = '(blank)|(choice)|(select)|(true)|(incorrect)|(correct)'
    mcq = '(choice)|(select)|(true)'

    try:

        
        
        if re.search(mcq, q_type):
            solve_mcq_questions(driver, question)
            
        elif 'blank' in q_type:
            solve_blank_questions(driver, question)
        check_alert(driver)

                # next_quest = driver.find_element(By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div/awd-question-button-bar/div/awd-next-button/button')
        next_quest = WebDriverWait(driver, 5).until(
            cond.element_to_be_clickable(
                (By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div/awd-question-button-bar/div/awd-next-button/button')
            )
        )
        next_quest.click()
        return False
    except:
        if not re.search(valid_types, q_type.lower()):
            print('Not a valid question type')
        else: 
            return True

In [17]:
global driver

driver = webdriver.Chrome(ChromeDriverManager().install())
USERNAME = os.getenv('CLIENT_USER')
PASSWORD = os.getenv('CLIENT_PASS')

'''
    Code that should login automatically, no longer works. I think the website was changed so it must be adjusted.
'''



# driver.get('https://accounts.mheducation.com/login?app=connect.mheducation.com&redirectUrl=https:%2F%2Fcaas.mheducation.com%2Fcaas%2Fheclr%2FlaunchConnect')

# input_username = WebDriverWait(driver, 20).until(
#     cond.element_to_be_clickable(
#         (By.XPATH, '//*[@id="login-email"]')
#     )
# )
# input_username.send_keys(USERNAME+'@student.gsu.edu')

# input_password = WebDriverWait(driver, 20).until(
#     cond.element_to_be_clickable(
#         (By.XPATH, '//*[@id="login-password"]')
#     )
# )
# input_password.send_keys(PASSWORD)

# signin = driver.find_element(By.XPATH, '//*[@id="login-submit-btn"]')
# signin.click()




/tmp/ipykernel_4839/2064709685.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [33]:
# start_questions = WebDriverWait(driver,20).until(
#     cond.element_to_be_clickable(
#         (By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div/awd-welcome-button-bar/button[2]/span[1]')
#     )
# )
# start_questions.click()


with open('current_mcq.text', 'w+') as mcq:
    mcq.truncate()
    mcq.close()
with open('current_blanks.text', 'a+') as bq:
    bq.truncate()
    bq.close()


Below is the main loop that will run continuously. To begin, select a chapter and get to a question that should be answered by the bot.

Questions that will not be answered:
    - match
    - Free response (not given by mcgraw anyway)
    - sometimes breaks on T/F questions, not sure why.

In [59]:
'''
mcq and valid_types contains keywords to search for in regex format, separated by parentheses.

try/except blocks used to continue looping without breaking. Usually the issue is resolved automatically if it can be answered.

'''

end_early = False
mcq = '(choice)|(select)|(true)'
valid_types = '(blank)|(choice)|(select)|(true)|(incorrect)|(correct)'

while not end_early:
    try:
        q_type = get_question_type(driver)
        question = get_prompt(driver)
        question = re.sub(r'(\nBlank\n)|(\n)', '', question) 
        
        q_type = q_type.lower()
        
        if re.search(mcq, q_type):
            solve_mcq_questions(driver, question)
        elif 'blank' in q_type:
            solve_blank_questions(driver, question)
        check_alert(driver)

        next_quest = WebDriverWait(driver, 5).until(
            cond.element_to_be_clickable(
                (By.XPATH, '//*[@id="root-container"]/div[2]/div/awd-main-container/div/awd-navigation-bar/div/awd-question-button-bar/div/awd-next-button/button')
            )
        )
        next_quest.click()
    
    except (ElementClickInterceptedException, StaleElementReferenceException, TimeoutException, NoSuchElementException) as e: 
        if not re.search(valid_types, q_type.lower()):
            print('Not a valid question type')
            end_early = True
        else:
            pass
            # end_early = answer_once(question, q_type)
    

KeyboardInterrupt: 